# Loading Document

Load a PDF into a sequence of `Document` objects.

The type of `Document` is `langchain_core.documents.Document`

通过这行代码，可以实现把 PDF 的每一页转换分别转换为一个`Document`对象：

```
docs = loader.load()
```

In [6]:
from langchain_community.document_loaders import PyPDFLoader
from uvloop.includes.stdlib import aio_wait

file_path = "../data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

# 每个一个 document 对象
print(len(docs))

107


In [13]:
print(docs[0].metadata, "\n\n")
print(docs[0].page_content[:200])
# dir(docs[0])

{'source': '../data/nke-10k-2023.pdf', 'page': 0} 


Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K
(Mark One)
☑  ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(D) OF THE SECURITIES EXCHANGE ACT OF 1934
F


# Splitting

不管取数据还是做问答，

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200,
                                               add_start_index=True)

all_splits = text_splitter.split_documents(docs)
print(len(all_splits))

516


In [34]:
split = all_splits[0]
print(split.metadata, "\n\n",
      len(split.page_content))

{'source': '../data/nke-10k-2023.pdf', 'page': 0, 'start_index': 0} 

 972


In [33]:
print(len(docs[0].page_content), "\t", len(split.page_content), "\n\n")

print(docs[0].page_content[900:972], "\n")
print(split.page_content[900:972])


3646 	 972 


each class) (Trading symbol) (Name of each exchange on which registered) 

each class) (Trading symbol) (Name of each exchange on which registered)


# Embeddings

把文本变成向量，然后可以进行向量搜索

In [39]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [40]:
vector_1 = embeddings.embed_query(all_splits[0].page_content)
vector_2 = embeddings.embed_query(all_splits[1].page_content)

assert len(vector_1) == len(vector_2)
print(f"Generated vectors of length {len(vector_1)}\n")
print(vector_1[:10])

Generated vectors of length 768

[0.047472357749938965, 0.021675849333405495, -0.009018078446388245, 0.005356733687222004, 0.025557702407240868, -0.010230264626443386, -0.008413944393396378, 0.03930392488837242, 0.02157050184905529, -0.024095406755805016]


# Vector Store

In [55]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

client = QdrantClient(":memory:")
collection_name = "test"
if not client.collection_exists(collection_name):
    client.create_collection("test", vectors_config=VectorParams(size=768, distance=Distance.COSINE))
vector_store = QdrantVectorStore(
    client=client,
    collection_name="test",
    embedding=embeddings,
)

ids = vector_store.add_documents(documents=all_splits)

In [51]:
print(len(ids))
print(ids[:3])

516
['899e7c374ca94fea95cd42a3eb2f79a2', 'd9564a67d3f542a2a7e5c7c9d52123a6', '706550bd7aa64cd4a90975f758bb8be1']


In [56]:
results = vector_store.similarity_search("How many distribution centers does Nike have in the US?")
print(len(results), "\n\n", results[0])

4 

 page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'source': '../data/nke-10k-2023.pdf', 'page': 

In [65]:
results = await vector_store.asimilarity_search("How many distribution centers does Nike have in the US?")
print(len(results), "\n\n", results[0])

4 

 page_content='operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater China geography, occupied by employees focused on implementing our
wholesale, NIKE Direct and merchandising strategies in the region, among other functions.
In the United States, NIKE has eight significant distribution centers. Five are located in or near Memphis, Tennessee, two of which are owned and three of which are
leased. Two other distribution centers, one located in Indianapolis, Indiana and one located in Dayton, Tennessee, are leased and operated by third-party logistics
providers. One distribution center for Converse is located in Ontario, California, which is leased. NIKE has a number of distribution facilities outside the United States,
some of which are leased and operated by third-party logistics providers. The most significant distribution facilities outside the United States are located in Laakdal,' metadata={'source': '../data/nke-10k-2023.pdf', 'page': 

In [66]:
results = vector_store.similarity_search_with_score("What was Nike's revenue in 2023?")
doc, score = results[0]
print(f"Score: {score}\n")
print(doc)

Score: 0.8137388363352561

page_content='Table of Contents
YEAR ENDED MAY 31,
(Dollars in millions) 2023 2022 2021
REVENUES
North America $ 21,608 $ 18,353 $ 17,179 
Europe, Middle East & Africa 13,418 12,479 11,456 
Greater China 7,248 7,547 8,290 
Asia Pacific & Latin America 6,431 5,955 5,343 
Global Brand Divisions 58 102 25 
Total NIKE Brand 48,763 44,436 42,293 
Converse 2,427 2,346 2,205 
Corporate 27 (72) 40 
TOTAL NIKE, INC. REVENUES $ 51,217 $ 46,710 $ 44,538 
EARNINGS BEFORE INTEREST AND TAXES
North America $ 5,454 $ 5,114 $ 5,089 
Europe, Middle East & Africa 3,531 3,293 2,435 
Greater China 2,283 2,365 3,243 
Asia Pacific & Latin America 1,932 1,896 1,530 
Global Brand Divisions (4,841) (4,262) (3,656)
Converse 676 669 543 
Corporate (2,840) (2,219) (2,261)
Interest expense (income), net (6) 205 262 
TOTAL NIKE, INC. INCOME BEFORE INCOME TAXES $ 6,201 $ 6,651 $ 6,661 
ADDITIONS TO PROPERTY, PLANT AND EQUIPMENT
North America $ 283 $ 146 $ 98 
Europe, Middle East & Africa 21

In [67]:
embedding = embeddings.embed_query("How were Nike's margins impacted in 2023?")

results = vector_store.similarity_search_by_vector(embedding)
print(results[0])

page_content='Table of Contents
GROSS MARGIN
FISCAL 2023 COMPARED TO FISCAL 2022
For fiscal 2023, our consolidated gross profit increased 4% to $22,292 million compared to $21,479 million for fiscal 2022. Gross margin decreased 250 basis points to
43.5% for fiscal 2023 compared to 46.0% for fiscal 2022 due to the following:
*Wholesale equivalent
The decrease in gross margin for fiscal 2023 was primarily due to:
• Higher NIKE Brand product costs, on a wholesale equivalent basis, primarily due to higher input costs and elevated inbound freight and logistics costs as well as
product mix;
• Lower margin in our NIKE Direct business, driven by higher promotional activity to liquidate inventory in the current period compared to lower promotional activity in
the prior period resulting from lower available inventory supply;
• Unfavorable changes in net foreign currency exchange rates, including hedges; and
• Lower off-price margin, on a wholesale equivalent basis.
This was partially offset by:'

# Retrievers

由于 Langchain 的 `Vector Store` 对象没有继承 `Runnable` 接口，但 `Retriever` 是 Runnable 类型的，所以需要把 Vector Store 转换为 Runnable 对象，再作为 Retriever 使用。

转换有两种常见方式：

1. @chain 装饰器
2. VectorStore 的 `as_retriever()` 方法

##  1. 使用装饰器

In [74]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import chain

@chain
def retriever(query: str) -> List[Document]:
    return vector_store.similarity_search(query, k=1)




results = retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)
res = results[1]
print(res[0].metadata, "\n\n", res[0].page_content[:100])


{'source': '../data/nke-10k-2023.pdf', 'page': 3, 'start_index': 0, '_id': '382ef9330fb54a59bc58fb3218ff668f', '_collection_name': 'test'} 

 Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws


In [72]:
from typing import List
from langchain_core.documents import Document
from langchain_core.vectorstores import VectorStore
from langchain_core.runnables import chain


@chain
def retriever2(params:dict[str, int, VectorStore]) -> List[Document]:
    query, k, store = params["query"], params["k"], params["store"]
    return store.similarity_search(query, k=k)




res = retriever2.invoke({
    "query": "How many distribution centers does Nike have in the US?",
    "k":1,
    "store": vector_store
})
print(res[0].metadata, "\n\n", res[0].page_content[:100])


{'source': '../data/nke-10k-2023.pdf', 'page': 26, 'start_index': 804, '_id': '6c33199bafc445c5a0ffd198a23f4bde', '_collection_name': 'test'} 

 operations. We also lease an office complex in Shanghai, China, our headquarters for our Greater Chi


## 使用 `as_retriever()`

In [75]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

results = retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)
res = results[1]
print(res[0].metadata, "\n\n", res[0].page_content[:100])

{'source': '../data/nke-10k-2023.pdf', 'page': 3, 'start_index': 0, '_id': '382ef9330fb54a59bc58fb3218ff668f', '_collection_name': 'test'} 

 Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws


In [80]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 1, "score_threshold":0.85},
)

results = retriever.batch(
    [
        "How many distribution centers does Nike have in the US?",
        "When was Nike incorporated?",
    ],
)
res = results[1]
if len(res) > 0:
    print(res[0].metadata, "\n\n", res[0].page_content[:100])

{'source': '../data/nke-10k-2023.pdf', 'page': 3, 'start_index': 0, '_id': '382ef9330fb54a59bc58fb3218ff668f', '_collection_name': 'test'} 

 Table of Contents
PART I
ITEM 1. BUSINESS
GENERAL
NIKE, Inc. was incorporated in 1967 under the laws


# 总结

实现一个语意检索引擎，包含以下步骤：

1. 通过 DocumentLoader 把文件转换为 langchain 的 `Document` 对象
2. 通过 `TextSplitter` 对象把 Document 切成小片
3. 创建 embedding 对象，用于把小片文本转换为向量，以便进行向量的相似性检索
4. 创建 vector store 对象，把小片文本以向量形式存入数据库
5. 创建 retriever 对象，用于检索
